In [1]:
import os
import librosa
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from tensorflow.image import resize

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:

model = tf.keras.models.load_model("/content/drive/My Drive/music_genre_classification/train_model.keras")
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 128, 128, 32)        │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 126, 126, 32)        │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 63, 63, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 61, 61, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 30, 30, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 28, 28, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 14, 14, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 12, 12, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 6, 6, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 6, 6, 512)           │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 4, 4, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 2, 2, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 2, 2, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1200)                │       2,458,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 1200)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │          12,010 │
└──────────────────────────────────────┴─────────────────────────────┴──────────────

 Total params: 21,547,376 (82.20 MB)

 Trainable params: 7,182,458 (27.40 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 14,364,918 (54.80 MB)

In [5]:
classes = ['blues','classical','country','disco','hiphop','jazz','metal','pop','reggae','rock']

In [6]:
def preprocess_file(file_path,target_shape=(128,128)):
  data = []
  audio_data,sr = librosa.load(file_path,sr=None)
  duration = 4
  overlap_duration = 2
  samples = duration * sr
  overlap_sample = overlap_duration * sr
  total_samples = max(1, int(np.ceil((len(audio_data) - samples) / (samples - overlap_sample))) + 1)

  for i in range(total_samples):
                    start = i * (samples - overlap_sample)
                    end = start + samples
                    chunk = audio_data[start:end]

                    # Ignore segments that are too short
                    if len(chunk) < 512:
                        continue

                    # Dynamically adjust n_fft to avoid warnings
                    n_fft = min(2048, len(chunk) // 2)

                    spectrogram = librosa.feature.melspectrogram(y=chunk, sr=sr, n_fft=n_fft)
                    spectrogram = resize(np.expand_dims(spectrogram, axis=-1), target_shape)

                    data.append(spectrogram)

  return np.array(data)

In [7]:
file_path = "/content/drive/My Drive/music_genre_classification/genres_original/metal/metal.00007.wav"
x_test = preprocess_file(file_path)
print("done")

done


In [9]:
x_test.shape

(15, 128, 128, 1)

In [14]:
def model_pred(x_test):
  y_pred = model.predict(x_test)
  pred_cat = np.argmax(y_pred,axis=1)
  unique_el,cnt = np.unique(pred_cat,return_counts=True)
  max_cnt = np.max(cnt)
  max_el = unique_el[cnt==max_cnt][0]
  return classes[max_el]

In [15]:
model_pred(x_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step


'metal'

In [16]:
print("Music genre: Model prediction ==>",model_pred(x_test))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Music genre: Model prediction ==> metal
